In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv


In [2]:
import pandas as pd
import numpy
df = pd.read_csv('/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv')
df

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
...,...,...,...
93,4373204,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,7807643,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,4352464,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,6521784,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [3]:
# preprocessing 

import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence,stopwords=False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]','',sentence)
    if stopwords:
        sentence = remove_stopwords(sentence)
    return sentence


def get_cleaned_sentences(df,stopwords=False):
    sents = df[['Questions']];
    cleaned_sentences = []
    
    for index,row in df.iterrows():
        cleaned = clean_sentence(row['Questions'],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences


clean_sentences = get_cleaned_sentences(df,stopwords=True)
print(clean_sentences)    
print('\n')
clean_sentences_noStopWords = get_cleaned_sentences(df,stopwords=False)
print(clean_sentences_noStopWords)    

['mean mental illness', 'mental illness affect', 'causes mental illness', 'warning signs mental illness', 'people mental illness recover', 'know appears symptoms mental disorder', 'mental health professional child', 'treatment options available', 'involved treatment need know', 'difference mental health professionals', 'mental health professional right child', 'involved treatment need know', 'help', 'know starting new medication', 'feel better taking medication mean cured stop taking', 'help paying medication', 'therapy', 'learn types mental health treatment', 'different types mental health professionals', 'support group', 'inpatient care', 'local services', 'learn information clinical trials', 'learn information creating psychiatric advance directive', 'mental health', 'support doesnt help', 'prevent mental health problems', 'cures mental health problems', 'causes mental health problems', 'im worried mental health', 'know im unwell', 'im worried friend relative', 'deal telling', 'need

In [4]:
# Word2Vec

from gensim.models import Word2Vec
import gensim.downloader as api

w2v_model = None
try :
    w2v_model = gensim.models.KeyedVectors.load('./w2vecmodel.mod')
    print("Loaded w2v model")
except:
    w2v_model = api.load('word2vec-google-news-300')
    w2v_model.save("./w2vecmodel.mod")
    print("Saved w2v model")
    
w2vec_emvedding_size = len(w2v_model['computer'])

[=================================================-] 99.9% 1660.8/1662.8MB downloaded
Saved w2v model


In [5]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
confidence = 0.75
def getWordVec(word,model):
    samp=model["computer"]
    vec = [0]*len(samp)
    try: 
        vec = model[word]
    except:
        vec = [0]*len(samp)
    return (vec)

def getPharseEmbedding(phrase,model):
    samp = getWordVec('computer',model)
    vec = numpy.array([0] * len(samp))
    for word in phrase.split():
        vec = vec + numpy.array(getWordVec(word,model))
        
    return vec.reshape(1,-1)


def retrieveAndPrintFAQ(question_embed,sentence_embed,FAQdf,sentences):
    max_sim = -1
    index_sim = -1
    for index,faq_embed in enumerate(sentence_embed):
        sim = cosine_similarity(faq_embed,question_embed)[0][0]
        if index < 4 :
            print(sim,FAQdf.iloc[index,1])
        if (sim > max_sim) and (sim > confidence):
            max_sim = sim
            index_sim = index
    print("\n")
    print("Question : ",question)
    print("\n")
    if index_sim != -1:
        print("Retrieved : ",FAQdf.iloc[index_sim,1])
        print("\n")
        print("Answer : ",FAQdf.iloc[index_sim,2])
    else:
        print("No match found")

In [6]:
question = "what's the definition of mental illness"
# question = "what's what what what"
# question = "What does it mean to not have mental illness?"
# question = "causes of mental illness"

In [7]:
sent_embeddings = []
for sent in clean_sentences:
    sent_embeddings.append(getPharseEmbedding(sent,w2v_model))

question_embedding = getPharseEmbedding(question,w2v_model)
retrieveAndPrintFAQ(question_embedding,sent_embeddings,df,clean_sentences)

0.7899959246133863 What does it mean to have a mental illness?
0.7203173546671922 Who does mental illness affect?
0.7416810059308135 What causes mental illness?
0.6227148624595191 What are some of the warning signs of mental illness?


Question :  what's the definition of mental illness


Retrieved :  What does it mean to have a mental illness?


Answer :  Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.
Mental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person rec

In [8]:
# https://huggingface.co/AnnaWegmann/Style-Embedding